In [1]:
import requests
import pandas as pd
import io
import operator

Downloading training dataset

In [2]:
url_train = requests.get("http://kevincrook.com/utd/market_basket_training.txt")
with io.open('train.csv','wb') as infile:
    infile.write(url_train.content)

Creating dataframe from training dataset

In [5]:
cols = ['f',2,3,4,5]
dataframe = pd.read_csv('train.csv', header=None, names = cols).fillna('')
train_data = dataframe.groupby([2,3,4,5]).count().reset_index()
train_data[6] = ''
train_data['set'] =  train_data[[2,3,4,5,6]].values.tolist()

Downloading testing dataset

In [6]:
url_test = requests.get("http://kevincrook.com/utd/market_basket_test.txt")
with io.open('test.csv','wb') as infile_test:
    infile_test.write(url_test.content)

Creating testing dataframe 

In [7]:
test_data = pd.read_csv('test.csv', header=None, names = cols).fillna('')
test_data['set'] = test_data[[2,3,4,5]].values.tolist()
r_df = test_data.copy()
r_df['Recommendation'] = ''

Creating recommenation dataset according to frequency of sets

In [8]:
for index_test, row_test in test_data.iterrows():
    r_freq = dict() 
    for index_train, row_train in train_data.iterrows(): 
        a = len(set(row_test['set'])) + 1
        b = len(set(row_train['set']))
        if a == b:
            if set(row_test['set']).issubset(set(row_train['set'])):
                r_freq[tuple(row_train['set'])] =  row_train['f']
    sorted_dict = sorted(r_freq.items(), key=operator.itemgetter(1), reverse=True)
    if sorted_dict != []:
        r_set = set(sorted_dict[0][0])
        recommended_item = r_set - set(row_test['set'])
        r_df['Recommendation'][index_test] = recommended_item
    else:
        test_list = list(set(row_test['set']))
        n_l = len(test_list)
        for i in range(0,n_l):
            l = test_list[:]
            del l[i]
            for index_train, row_train in train_data.iterrows():
                n_a = len(set(l)) + 1
                n_b = len(set(row_train['set']))
                if n_a == n_b:
                    if set(l).issubset(set(row_train['set'])):
                        r_freq[tuple(row_train['set'])] =  row_train['f']
        sorted_dict = sorted(r_freq.items(), key=operator.itemgetter(1), reverse=True)
        if sorted_dict != []:
            r_set = set(sorted_dict[0][0])
            recommended_item = r_set - set(row_test['set'])
            r_df['Recommendation'][index_test] = recommended_item
        else:
            test_list = list(set(row_test['set']))
            n_l = len(test_list)
            for i in range(0,n_l):
                l = test_list[i:i+1]
                l.append('')
                for index_train, row_train in train_data.iterrows():
                    n_a = len(set(l)) + 1
                    n_b = len(set(row_train['set']))
                    if n_a == n_b:
                        if set(l).issubset(set(row_train['set'])):
                            r_freq[tuple(row_train['set'])] =  row_train['f']
            sorted_dict = sorted(r_freq.items(), key=operator.itemgetter(1), reverse=True)
            if sorted_dict != []:
                r_set = set(sorted_dict[0][0])
                recommended_item = r_set - set(row_test['set'])
                r_df['Recommendation'][index_test] = recommended_item
            else:
                r_df['Recommendation'][index_test] = 'No Recommendation'

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Saving final dataset to file according to requirement

In [9]:
final_df = r_df[['f','Recommendation']]
final_df['Recommendation'] = final_df['Recommendation'].apply(lambda x: ''.join(str(e) for e in x))
final_df['f'] = final_df['f'].apply(lambda x: str(x).zfill(3))
final_df.to_csv("market_basket_recommendations.txt", sep=',', encoding = 'utf-8',index=False,header=False)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
